# HoroRF
> Comparing two hyperbolic RF methods

In [2]:
%load_ext autoreload
%autoreload 2

In [9]:
from HoroRF.hororf.rf_trainer import run_train

In [13]:
%%bash
cd HoroRF
/home/phil/mambaforge/envs/hdt/bin/python train_hyp_rf.py -h

tar (child): ./logs/output__/code.tar.gz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: ./logs/output__/code.tar.gz: Cannot write: Broken pipe
tar: Child returned status 2
tar: Error is not recoverable: exiting now
2023-09-12 08:54:09,911 [hororf.rf_trainer] 1 GPUs available
2023-09-12 08:54:09,911 [hororf.rf_trainer] Using seed 17 on class 2
2023-09-12 08:54:09,920 [hororf.utils] 34 datapoints in dataset 'datasets.karate'
2023-09-12 08:54:09,920 [hororf.utils] 34 datapoints in test dataset 'datasets.karate'
2023-09-12 08:54:09,939 [hororf.rf_trainer] 17 train and 17 test samples for fold 0
2023-09-12 08:54:13,315 [hororf.rf_trainer] Hyperbolic tree f1 micro: 0.9412, f1 macro: 0.9412, AUPR: 0.0000. Mean depth of 1.00
2023-09-12 08:54:15,614 [hororf.rf_trainer] 17 train and 17 test samples for fold 1
2023-09-12 08:54:18,124 [hororf.rf_trainer] Hyperbolic tree f1 micro: 1.0000, f1 macro: 1.0000, AUPR: 0.0000. Mean depth of 3.00


In [1]:
%%bash
cd HoroRF
/home/phil/mambaforge/envs/hdt/bin/python train_hyp_rf.py -h

# NeuroSEED version

tar (child): ./logs/output_/code.tar.gz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: ./logs/output_/code.tar.gz: Cannot write: Broken pipe
tar: Child returned status 2
tar: Error is not recoverable: exiting now
2023-09-12 09:52:24,360 [hororf.rf_trainer] 1 GPUs available
2023-09-12 09:52:24,360 [hororf.rf_trainer] Using seed 17 on class 2
2023-09-12 09:52:28,224 [hororf.utils] 800 datapoints in dataset 'datasets.neuroseed'
2023-09-12 09:52:28,224 [hororf.utils] 200 datapoints in test dataset 'datasets.neuroseed'
/home/phil/mambaforge/envs/hdt/lib/python3.11/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(
2023-09-12 09:52:28,251 [hororf.rf_trainer] 400 train and 400 test samples for fold 0
2023-09-12 09:53:18,410 [hororf.rf_trainer] Hyperbolic tree f1 micro: 0.6700, f1 macro: 0.2155, AUPR: 0.0000. Mean depth of 6.00
/home